<a href="https://colab.research.google.com/github/Datamin-D/movie_predict/blob/master/Movie_predict_Webcrawling_mojo_cal%3Eimdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#라이브러리, 셀레니움 설치
#import
import requests
from bs4 import BeautifulSoup
import time
from random import randint
import re
import pandas as pd
import pickle
 
#installing selenium, prepare to connect 
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
 
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

     |████████████████████████████████| 911kB 8.6MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:7 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 htt

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: use options instead of chrome_options


1. df 그때그때  수정하도록 바꾸기.
2. 오류 수정.
3. feauture film 숫자 설명.

In [0]:
#Boxoffice mojo > extract movie titles by years(skip pickle has the movie titles info) 

Columns = ["Movie_title","Genre","Rate","Rating_count","Run_time","Budget","Gross_USA","Opening Weekend USA","Cumulative Worldwide Gross","Language","Directors","Writers","Stars","Certificate","Released_date","Domestic_distributor"]

#DataFrame생성  19개. *** 안바쁠 때 이쁘게 바꿔보기.
df0 = pd.DataFrame(columns=Columns)
df1 = pd.DataFrame(columns=Columns)
df2 = pd.DataFrame(columns=Columns)
df3 = pd.DataFrame(columns=Columns)
df4 = pd.DataFrame(columns=Columns)
df5 = pd.DataFrame(columns=Columns)
df6 = pd.DataFrame(columns=Columns)
df7 = pd.DataFrame(columns=Columns)
df8 = pd.DataFrame(columns=Columns)
df9 = pd.DataFrame(columns=Columns)
df10 = pd.DataFrame(columns=Columns)
df11 = pd.DataFrame(columns=Columns)
df12 = pd.DataFrame(columns=Columns)
df13 = pd.DataFrame(columns=Columns)
df14 = pd.DataFrame(columns=Columns)
df15 = pd.DataFrame(columns=Columns)
df16 = pd.DataFrame(columns=Columns)
df17 = pd.DataFrame(columns=Columns)
df18 = pd.DataFrame(columns=Columns)
df19 = pd.DataFrame(columns=Columns)


#error counter -->어디서 정보 소실이 일어난 찾아내기
fail_to_fetch_indv_movie = 0 # 영화 개별페이지 접근 실패.
count_ab_genre = 0 #제외할 장르(short 등)
see_more_credits_error = 0

# 수집할 영화 연도 범위
released_years = []
released_months = []

years = [released_years.append(i) for i in range(2019,2020)] #연 #실험용 데이터
months = [released_months.append(i) for i in range(12,13)] #월
#-->연 단위로 수집할 수 있게 해보자. 20년.
# raw data 수집,parsing, 영화 title저장

for ind_year in range(len(released_years)):
  save_file_name = "movie_data_"+str(released_years[ind_year])+".csv" #제목
  print(save_file_name,"수집 중!!")
  df_name = [df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19][ind_year]
  df_name
  df_row = -1 #df 저장할때 loc 위치로 쓸 것.
  for ind_month in range(len(released_months)):
    # . mojo calender 사이트로 이동
    mojo_cal_url = "https://www.boxofficemojo.com/calendar/"+str(released_years[ind_year])+"-"+str(released_months[ind_month])+"-01/"
    wd.get(mojo_cal_url) #mojo calender 해당 연월 열기
    time.sleep(randint(1,2)) 

# 1. mojo calender -각 월의  영화 컨테이너 목록 
    mojo_yymm_list = wd.find_elements_by_css_selector("div.a-section.a-spacing-none.mojo-schedule-release-details")

# 2. mojo_yymm_list 안의 영화 하나하나씩 다운로드. ****문제발생지점
    for mojo_movie in mojo_yymm_list:
        df_row = df_row+1 #df에 저장할 열. 연도마다 -1로 초기화됨. ***무시하셔도 됩니다!
        try:
          movie_title = mojo_movie.find_element_by_css_selector("a.a-link-normal") #컨테이너 안의 영화 클릭 ****문제발생지점
          movie_title.click()
          time.sleep(randint(2,4))
        except:
            pass
    # 3.title summary클릭, 영화 이름 저장(나중 except 처리 위해서. 1.imdb이동 2.movie_title 못 찾을 때)
        title_summary = wd.find_element_by_css_selector("#title-summary-refiner > a > span")
        title_summary.click()
        time.sleep(randint(2,4))  
        mojo_url = wd.current_url
        m_title = wd.find_element_by_css_selector("#a-page > main > div > div.a-section.a-spacing-none.mojo-summary > div.a-section.mojo-heading-summary > div > div > div.a-fixed-left-grid-col.a-col-right > div > h1").text #mojo>cal>title summary화면에서 제목 저장
    # 4. movie_code 수집
        try:
        target = str(mojo_url)
        p = re.compile(r"(https://www.boxofficemojo.com/title/)([a-zA-z]{2}[0-9]{7})(/\?ref_=bo_rl_ti)")
        m = p.match(target)
        movie_code = m.group(2) #영화코드수집 --완료
        wd.close #mojoboxoffice 닫기
    # 5.imdb로 이동,열기 (닫는 곳도 확인)
        imdb_indv_url = "https://www.imdb.com/title/" + movie_code + "/?ref_=fn_al_tt_1"
        wd.get(imdb_indv_url) #개별 imdb영화 페이지 열기.
        time.sleep(randint(2,4))
        # 여기부터 영화 개별 웹페이지 진입 성공, 자료수집 시작.
        except:
        url = "https://www.imdb.com/find?q="+str(m_title)+"&ref_=nv_sr_sm" #movie_title에 영화 이름 1개 들어가야함. 
        wd.get(url) #개별영화 페이지 열기
        time.sleep(randint(2,4))
        #print(wd.page_source)  # results
        try:
            click_movie = wd.find_elements_by_css_selector('td.result_text > a') #영화 검색창 후 이름 클릭
            click_movie[0].click() #클릭이 안되는 경우, no result found. pass하게 코딩.
        
            time.sleep(randint(1,3))
        except IndexError: #영화 정보가 없어서 클릭이 안되면.
            fail_to_get_indv_movie = fail_to_get_indv_movie + 1
            continue 
        # 여기부터 영화 개별 웹페이지 진입 성공, 자료수집  
      """
      #제외할 영화(imdb영화 개별 페이지 인 상태.)
      try:
        targets = wd.find_elements_by_css_selector("#title-overview-widget > div.vital > div.title_block > div > div.titleBar > div.title_wrapper  > div > a")
        abandoned_genre = ["Short","Video game","Episode","TV Movie","TV Special","TV Mini-Series","TV Movie","TV Series","Video","TV Series","Talk-Show"] #제외할 데이터타입
        

        def ab_collector():
          for target in targets:
            time.sleep(1)
            for ab in abandoned_genre:
              time.sleep(1)
              if ab.lower() in target.text.lower(): #lower() 괄호 꼭 붙여줘야함.
                return True #함수는 return을 만나는 순간 결과값 돌려주고 함수 빠져나감... 코딩 주의

        if ab_collector():
          count_ab_genre = count_ab_genre + 1
          continue
        else:
          pass

      except:
        pass  

      #Movie_title 
      Movie_title = []
      try:
        imdb_title = wd.find_element_by_css_selector("#title-overview-widget > div.vital > div.title_block > div > div.titleBar > div.title_wrapper > h1").text
        Movie_title = imdb_title
      except NoSuchElementException:
        Movie_title.append(m_title)    

      
    #Genre - 수집 중 날짜까지 리스트 마지막에 수집되어서 index[-1]지우는 식으로 코딩함. 성공!
      Genre = []
      try:
        genre = wd.find_elements_by_css_selector("div.subtext a")
        for gen in range(len(genre)-1):
          Genre.append(genre[gen].text)
      except:
        Genre = "NA"

    #Rate(전체 별점) 
      Rate = []
      try:
        Rate.append(wd.find_element_by_css_selector("div.ratingValue span:nth-of-type(1):first-child").text) # nth-of-type(n번째) n은 1부터 시작, first-child하면 부모 모든 요소 중 첫번째 자식만 선택 
      except NoSuchElementException:
        Rate.append("NA")
    #Rating_Count
      Rating_count = "NA"
      try:
        Rating_count = wd.find_element_by_css_selector("#title-overview-widget > div.vital > div.title_block > div > div.ratings_wrapper > div.imdbRating > a > span").text
      except:
        Rating_count = "NA"

    #Run_time
      Run_time = []
      try:
        Run_time.append(wd.find_element_by_css_selector('#title-overview-widget > div.vital > div.title_block > div > div.titleBar > div.title_wrapper > div > time').text)
      except NoSuchElementException:
        Run_time.append("")

      

    #선택자#titleDetails >.txt-block
      Bcontainer = wd.find_elements_by_css_selector("#titleDetails >.txt-block")
  #Budget 
      Budget = "NA"
      try:
        for ind_Budget in range(len(Bcontainer)):
          if "Budget:" in Bcontainer[ind_Budget].text: #if문에 해당하는 내용이 없다해서 오류가 일어나지는 않음.
            budget = Bcontainer[ind_Budget].text
            Budget = budget.split(":")[1]
            
            break  #break는 조건이 참이면 반복문 종료. cf)continue는 아래의 코드를 실행않고 다음 반복으로 넘어감.
          else:
            Budget = "NA"
      except:
        Budget = "NA"
  #Gross_USA 
      Gross_USA = "NA"
      try:
        for ind_gusa in range(len(Bcontainer)):
          if "Gross USA:" in Bcontainer[ind_gusa].text:
            gusa = Bcontainer[ind_gusa].text
            Gross_USA = gusa.split(":")[1]
            break
          else:
            Gross_USA = "NA"
      except:
        Gross_USA = "NA"
  #Opening Weekend USA 
      Opening_Weekend_USA  = "NA"
      try:
        for ind_OWU in range(len(Bcontainer)):
          if "Opening Weekend USA:" in Bcontainer[ind_OWU].text:
            OWU = Bcontainer[ind_OWU].text
            Opening_Weekend_USA = OWU.split(":")[1]
            break
          else:
            Opening_Weekend_USA = "NA"
      except:
        Opening_Weekend_USA = "NA"

  #Cumulative Worldwide Gross 
      Cumulative_Worldwide_Gross  = "NA"
      try:
        for ind_CWG in range(len(Bcontainer)):
          if "Cumulative Worldwide Gross:" in Bcontainer[ind_CWG].text:
            CWG = Bcontainer[ind_CWG].text
            Cumulative_Worldwide_Gross = CWG.split(":")[1]
            break
          else:
            Cumulative_Worldwide_Gross = "NA"
      except:
        Cumulative_Worldwide_Gross = "NA"
      
    #Language: 로 시작하는 컨테이너찾기, 적당히 인덱싱해서 Language찾기. 
      Language = []
      try:
        find_language = wd.find_elements_by_css_selector("#titleDetails > div.txt-block")
        for ind_lang in range(len(find_language)):
          if find_language[ind_lang].text.strip().startswith("Language:"):
            Lang = find_language[ind_lang].text.strip()
        Language =  Lang[9:].strip().split("|")
        
      except NoSuchElementException:
        Language.append("NA")


      wd.close #imdb개별 영화 페이지 닫기

    #Certificate 급히 수정*** try except,continue
      Certificate = []
      #indv_movie_url는  개별 영화 웹페이지 url
      try:
        #Certificate > clicking see more button using Selenium
        url = "https://www.imdb.com/title/"+movie_code+"/parentalguide?ref_=tt_stry_pg#certification" #개별영화 페이지 movie_title에 영화 이름 1개 들어가야함. 
        wd.get(url) #certificate 페이지 열기
        time.sleep(randint(1,3))
        #컨테이너 li.ipl-inline-list__item a
        countries = wd.find_elements_by_css_selector("tr#certifications-list > td > ul.ipl-inline-list >li.ipl-inline-list__item > a")
        try:
          for c_ind in range(len(countries)):
            if "United States" in countries[c_ind].text:
              US_cert = countries[c_ind].text
              nation_cert = US_cert.split(":")
              Certificate.append(nation_cert[1])
            else:
              pass #다른 나라들은 pass. 
        except:
          Certificate.append("NA")
        #print(Certificate)
      except:
        Certificate.append("NA")
      wd.close #certificate 페이지 닫기
      #Released_date imdb에서 가져오기.


      furl = "https://www.imdb.com/title/"+movie_code+"/releaseinfo?ref_=tt_ov_inf"
      wd.get(furl)#release info 열기
      time.sleep(randint(1,2))
      #개봉한 나라 갯수 (=행 갯수)
      release_rows = wd.find_elements_by_css_selector("#releaseinfo_content > table.ipl-zebra-list.ipl-zebra-list--fixed-first.release-dates-table-test-only > tbody > tr > td.release-date-item__country-name > a") #개봉한 나라 이름 선택
      #print(len(release_rows)) 개봉한 나라 갯수 - 성공
      def cond_empty(row):
        try:
          wd.find_element_by_css_selector("#releaseinfo_content > table.ipl-zebra-list.ipl-zebra-list--fixed-first.release-dates-table-test-only > tbody > tr:nth-child("+ str(row + 1) +") > td.release-date-item__attributes--empty").text
          return True
        except:
          return False #성공 row12로 놓고해봄.
      def cond_limited(row):
        try:
          limited_selector = "#releaseinfo_content > table.ipl-zebra-list.ipl-zebra-list--fixed-first.release-dates-table-test-only > tbody > tr:nth-child(" + str(row + 1) + ") > td.release-date-item__attributes"
          if wd.find_element_by_css_selector(limited_selector).text.startswith("(limited)"):#상영이 limited
            return True
        except:
          return False


      # 개봉일자가 있는경우(전국,limited)
      try:
        for row in range(len(release_rows)-1): #각 행마다 검사 + 나라이름있는 행의 row 기억,해당 행이 USA로 시작,3번째 값이 비면 2번째 값 찾기 
          nation_selector= "#releaseinfo_content > table.ipl-zebra-list.ipl-zebra-list--fixed-first.release-dates-table-test-only > tbody > tr:nth-child("+ str(row+1) +") > td.release-date-item__country-name" #국가 이름 선택자
          cond_USA = wd.find_element_by_css_selector(nation_selector).text.startswith("USA") #개봉국가가 USA
          released_date_selector = "#releaseinfo_content > table.ipl-zebra-list.ipl-zebra-list--fixed-first.release-dates-table-test-only > tbody > tr:nth-child("+ str(row+1) + ") > td.release-date-item__date" #중간 칸에 있는 released date
          if cond_USA and cond_empty(row): #US & 전국개봉
            Released_date = wd.find_element_by_css_selector(released_date_selector).text
          else: #US& limited개봉
            if cond_USA and cond_limited(row):
              Released_date = wd.find_element_by_css_selector(released_date_selector).text
          #American Animals (2018)의 경우 limited, 전국 개봉 둘다 있는데 뭐로 수집했는지 확인하기. + 왜 NA로 떴나 문제해결.

      #개봉일자가 없는 경우 /오류 일어난 경우
      except NoSuchElementException:
        Released_date = "NA"

      wd.close #mojo-release info등 닫기
      time.sleep(randint(1,2))

      #Domestic Distributor 도 수집.
      durl = "https://www.boxofficemojo.com/title/" + movie_code+ "/?ref_=bo_se_r_1"
      wd.get(durl)
      Domestic_distributor = "NA"
      find_distributor = wd.find_elements_by_css_selector("#a-page > main > div > div.a-section.a-spacing-none.mojo-gutter.mojo-summary-table > div.a-section.a-spacing-none.mojo-summary-values.mojo-hidden-from-mobile > div.a-section.a-spacing-none")
      try:
        for ind_distributor in  find_distributor: #domestic distributor행 찾기
          if ind_distributor.text.startswith("Domestic Distributor"):
            distributor_line = ind_distributor.text
            target = distributor_line
            p = re.compile(r'(Domestic Distributor)\s([\'\",/.a-zA-z0-9_ :!?\+\-\&\#\$\@\%\-\=\^\`\(\)]*)') #배급사 이름 추리기
            m = p.match(target) #match는 처음부터 매칭해보고 없으면 진행 안함. search는 문자열 전체를 검색.(앞이 매칭 안되어도.)
            Domestic_distributor = m.group(2)
          else:
            pass
      except:
        Domestic_distributor = "NA"
      wd.close #domestic distributor info 닫기

    #See more credits접근 try except처리.
      try:
        url = "https://www.imdb.com/title/"+movie_code+"/fullcredits/?ref_=tt_ov_st_sm"
        wd.get(url) #See full cast & crew 페이지 열기
        time.sleep(randint(1,2))
        cu = wd.current_url
      except:
        see_more_credits_error = see_more_credits_error + 1
        pass 
        

    #Directors
      Directors_ch = []
      Directors = []
      try:
        director = wd.find_elements_by_css_selector("#fullcredits_content > table:nth-child(2) > tbody>tr")
        for ind_directors in range(len(director)):
          Directors_ch.append(director[ind_directors].text)
        for ind_Directors in range(len(Directors_ch)):
          target = Directors_ch[ind_Directors]
          p = re.compile("[\-. a-zA-z0-9]+")
          m = p.match(target)
          
          Director = m.group().strip()
          Directors.append(Director) #director_list
        #print(Directors) 
      except AttributeError:
        Directors.append("NA")
      except:
        Directors.append("NA")

    # Writers
      Writers = []
      try:
        Writing_credits = wd.find_elements_by_css_selector("#fullcredits_content > table:nth-child(4) > tbody > tr > td.name > a") #Writing Credits 박스 접근. 아래 자식으로 작가 이름 크롤링ㄱ 
        for ind_writer in range(len(Writing_credits)):
          Writers.append(Writing_credits[ind_writer].text)
      except:
        Writers = "NA"

      #if #fullcredits_content h4.dataHeaderWithBorder == Writing Credits 거기 멈춰서 안에 있는 박스 > ..는 복잡.
      #걍 nth-ofchild 기능으로 선택자 처리함.

      # Stars 상위16명까지. -->ㅡmin (2003)시도 완료
      #홀수 번호 배우들 수집 
      Stars_ch = []
      Stars_odd = []
      Stars_even = []
      Stars = []

      try:
        container_odd = wd.find_elements_by_css_selector("#fullcredits_content > table.cast_list > tbody > tr.odd") #홀수번호 배우들 데이터 수집
        for ind_oddstar in range(9): #8번째 홀수까지 반복, 15
          Stars_odd.append(container_odd[ind_oddstar].text)
      except IndexError: 
        pass

      #짝수 번호 배우들 수집
      try:
        container_even = wd.find_elements_by_css_selector("#fullcredits_content > table.cast_list > tbody > tr.even") #짝수번호 배우들 데이터 수집
        for ind_evenstar in range(9): #8번째 짝수까지 반복,16 
          Stars_even.append(container_even[ind_evenstar].text)
      except IndexError: 
        pass
      #홀,짝 합치기
      try:

        for ind_star in range(8):
          Stars_ch.append(Stars_odd[ind_star])
          Stars_ch.append(Stars_even[ind_star])
        #print(Stars_ch) 합치기 완료
      except IndexError: #배우 15명 미만일 시 pass.
        pass

      
      #배우"...캐릭터"지우기는 정규 표현식으로 처리.
      try:
        for ind_ch in range(len(Stars_ch)):
          target = Stars_ch[ind_ch]
          p = re.compile(r"([\-\.\' a-zA-z0-9]+)( ... )([\-\.\' a-zA-z0-9]+)")
          m = p.match(target)
          Star = m.group(1).strip()
          Stars.append(Star) #배우목록
          #print(Star)
      except AttributeError:
        Stars = Stars_ch
      wd.close ##See full cast & crew 페이지 닫기
      #loc은 행 데이터 채우기.
      #print(Movie_title,Genre,Rate,Run_time,Budget,Gross_USA,Cumulative_Worldwide_Gross,Opening_Weekend_USA,Language,Directors,Writers,Stars,Certificate)
      wd.get(mojo_cal_url)
      df_name.loc[df_row] = Movie_title,Genre,Rate,Rating_count,Run_time,Budget,Gross_USA,Opening_Weekend_USA,Cumulative_Worldwide_Gross,Language,Directors,Writers,Stars,Certificate, Released_date,Domestic_distributor #i번째 영화, 그에 해당하는 데이터
      time.sleep(randint(1,2))
  df_name.to_csv(save_file_name) #unit_titles만큼 모아서 하나로 저장
  time.sleep(randint(2,4))
  print(Movie_title,Genre,Rate,Rating_count,Run_time,Budget,Gross_USA,Opening_Weekend_USA,Cumulative_Worldwide_Gross,Language,Directors,Writers,Stars,Certificate, Released_date,Domestic_distributor)
  print(fail_to_fetch_indv_movie,count_ab_genre ,see_more_credits_error)
  #전처리할 때 중복되는 영화 지우기. 
  """

IndentationError: ignored

In [0]:
no1 = [1,2,3]
no2 = [29,30,31]
for ind_months in range(len(no1)):
  df_row = -1
  for ind_days in range(len(no2)):
    list1 = ["movie1","movie2","movie3"]
    for l in list1:
      df_row = df_row+1 #df에 저장할 열. 월마다 초기화해야.
      print("df_row",df_row) 
      print("movie",l,"ind_month:",no1[ind_months],"ind_days",no2[ind_days]) 

df_row 0
movie movie1 ind_month: 1 ind_days 29
df_row 1
movie movie2 ind_month: 1 ind_days 29
df_row 2
movie movie3 ind_month: 1 ind_days 29
df_row 3
movie movie1 ind_month: 1 ind_days 30
df_row 4
movie movie2 ind_month: 1 ind_days 30
df_row 5
movie movie3 ind_month: 1 ind_days 30
df_row 6
movie movie1 ind_month: 1 ind_days 31
df_row 7
movie movie2 ind_month: 1 ind_days 31
df_row 8
movie movie3 ind_month: 1 ind_days 31
df_row 0
movie movie1 ind_month: 2 ind_days 29
df_row 1
movie movie2 ind_month: 2 ind_days 29
df_row 2
movie movie3 ind_month: 2 ind_days 29
df_row 3
movie movie1 ind_month: 2 ind_days 30
df_row 4
movie movie2 ind_month: 2 ind_days 30
df_row 5
movie movie3 ind_month: 2 ind_days 30
df_row 6
movie movie1 ind_month: 2 ind_days 31
df_row 7
movie movie2 ind_month: 2 ind_days 31
df_row 8
movie movie3 ind_month: 2 ind_days 31
df_row 0
movie movie1 ind_month: 3 ind_days 29
df_row 1
movie movie2 ind_month: 3 ind_days 29
df_row 2
movie movie3 ind_month: 3 ind_days 29
df_row 3
movi

In [0]:
stores = driver.find_elements_by_css_selector("div.lsnx")
for store in stores:
    # 세부 데이터 수집
    name = store.find_element_by_css_selector("dt > a").text
    addr = store.find_element_by_css_selector("dd.addr").text
    phone = store.find_element_by_css_selector("dd.tel").text

In [5]:
year = [2010,2011]
col1 = ["name","$"]
name = ["min","ingpa"]
dollars = [111,222]
row = 0
for ind_year in range(len(year)):
  df_name = ["df0","df1"][ind_year]
  df_name = pd.DataFrame(columns = col1)
  n = name[ind_year]
  d = dollars[ind_year]
  df_name.loc[row] = n,d
  row = row+1
  print(df_name)

  name    $
0  min  111
    name    $
1  ingpa  222
